In [36]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [117]:
import pmdarima as pmd

def arimamodel(timeseriesarray):
    autoarima_model = pmd.auto_arima(timeseriesarray, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              trace=True)
    return autoarima_model

In [3]:
df1 = pd.read_csv(r'TTS_LBNL_public_file_19-Oct-2020_p1.csv') 
df2 = pd.read_csv(r'TTS_LBNL_public_file_19-Oct-2020_p2.csv',encoding="ISO-8859-1") 
df = df1.append(df2)

In [63]:
install_df = df[['installation_date','system_size_DC','total_installed_price','rebate_or_grant','zip_code', 'city', 'state','utility_service_territory']]
install_df['installation_date'] = pd.to_datetime(install_df['installation_date'])
install_df['year'] = pd.DatetimeIndex(install_df['installation_date']).year
install_df = install_df[install_df.total_installed_price != -9999]
install_df = install_df.groupby(['year','state']).agg({'total_installed_price':'mean'})
install_df = pd.pivot_table(install_df, values='total_installed_price', index=['year'], columns=['state'])
install_df = install_df.apply(lambda x: x.fillna(x.mean()),axis=0)
install_df = install_df.drop(columns=['OR','DE'])

In [5]:
zip_codes = pd.read_csv(r'zip_code_database.csv') 
zip_codes["zip"] = zip_codes["zip"].astype(str)

In [6]:
tot_sales_by_state = pd.read_excel (r'Sales to Ultimate Customers (Megawatthours) by State.xlsx')

In [7]:
avg_cost_by_state = pd.read_excel (r'Average Price (Cents per kilowatthour) by State.xlsx')

In [130]:
res_sales_df = pd.pivot_table(tot_sales_by_state, values='Residential', index=['State'], columns=['Year']).T
res_sales_df = res_sales_df.drop(columns=['US'])

In [159]:
res_avg_cost_df = pd.pivot_table(avg_cost_by_state, values='Residential', index=['State'], columns=['Year']).T
res_avg_cost_df = res_avg_cost_df.drop(columns=['US'])

In [120]:
def predict_install_cost():
    df = pd.DataFrame()
    states = install_df.columns

    for state in states:
        x = install_df[state]
        for i in range(11):
            model = ARIMA(x, order=(2,0,0))
            model_fit = model.fit()
            yhat = model_fit.predict(len(x), len(x))
            x = x.append(yhat)
            y = pd.Series(x, name=state).to_frame()
        df = df.join(y, how='outer')
    df.rename(index={22: 2020}, inplace=True)
    df.rename(index={23: 2021}, inplace=True)
    df.rename(index={24: 2022}, inplace=True)
    df.rename(index={25: 2023}, inplace=True)
    df.rename(index={26: 2024}, inplace=True)
    df.rename(index={27: 2025}, inplace=True)
    df.rename(index={28: 2026}, inplace=True)
    df.rename(index={29: 2027}, inplace=True)
    df.rename(index={30: 2028}, inplace=True)
    df.rename(index={31: 2029}, inplace=True)
    df.rename(index={32: 2030}, inplace=True)
    
    df = df.round(2).T
    df['Rank'] = df[2030].rank()
    df['Weight Score'] = df['Rank']*.3

    return df.sort_values('Rank', ascending=True).reset_index()

In [131]:
def predict_usage(sector_df):
    df = pd.DataFrame()
    states = sector_df.columns

    for state in states:
        x = sector_df[state]
        for i in range(11):
            model = ARIMA(x, order=(1, 2, 0))
            model_fit = model.fit()
            yhat = model_fit.predict(len(x), len(x), typ='levels')
            x = x.append(yhat)
            y = pd.Series(x, name=state).to_frame()
        df = df.join(y, how='outer')
    df.rename(index={30: 2020}, inplace=True)
    df.rename(index={31: 2021}, inplace=True)
    df.rename(index={32: 2022}, inplace=True)
    df.rename(index={33: 2023}, inplace=True)
    df.rename(index={34: 2024}, inplace=True)
    df.rename(index={35: 2025}, inplace=True)
    df.rename(index={36: 2026}, inplace=True)
    df.rename(index={37: 2027}, inplace=True)
    df.rename(index={38: 2028}, inplace=True)
    df.rename(index={39: 2029}, inplace=True)
    df.rename(index={40: 2030}, inplace=True)
    
    df = df.round(2).T
    df['Rank'] = df[2030].rank()
    df['Weight Score'] = df['Rank']*.2

    return df.sort_values('Rank', ascending=True).reset_index()

In [138]:
def predict_cost(sector_df):
    df = pd.DataFrame()
    states = sector_df.columns

    for state in states:
        x = sector_df[state]
        for i in range(11):
            model = ARIMA(x, order=(0,2,1))
            model_fit = model.fit()
            yhat = model_fit.predict(len(x), len(x), typ='levels')
            x = x.append(yhat)
            y = pd.Series(x, name=state).to_frame()
        df = df.join(y, how='outer')
    df.rename(index={30: 2020}, inplace=True)
    df.rename(index={31: 2021}, inplace=True)
    df.rename(index={32: 2022}, inplace=True)
    df.rename(index={33: 2023}, inplace=True)
    df.rename(index={34: 2024}, inplace=True)
    df.rename(index={35: 2025}, inplace=True)
    df.rename(index={36: 2026}, inplace=True)
    df.rename(index={37: 2027}, inplace=True)
    df.rename(index={38: 2028}, inplace=True)
    df.rename(index={39: 2029}, inplace=True)
    df.rename(index={40: 2030}, inplace=True)
    
    df = df.round(2).T
    df['Rank'] = df[2030].rank(ascending=False)
    df['Weight Score'] = df['Rank']*.5

    return df.sort_values('Rank', ascending=True).reset_index()

In [139]:
res_sales_prediction = predict_usage(res_sales_df)
res_cost_prediction = predict_cost(res_avg_cost_df)
installation_cost = predict_install_cost()

In [51]:
def map_zip(zip_codes):
    if len(zip_codes) == 3:
        return "00"+zip_codes
    elif len(zip_codes) == 4:
        return "0"+zip_codes
    else:
        return zip_codes

In [16]:
###Create seperate time series line chart for each state###

In [154]:
def YoY_ranking(df):
    temp_df = pd.DataFrame([])
    temp_df['index'] = df['index']
    for col in range(1998,2030):
        temp_df[col] = df[col].rank()
    
    return temp_df

In [155]:
YoY_Installation_Ranking = YoY_ranking(installation_cost)
YoY_Energy_Sales_Ranking = YoY_ranking(res_sales_prediction)
YoY_Energy_Cost_Ranking = YoY_ranking(res_cost_prediction)

In [156]:
installation_plot = px.line(YoY_Installation_Ranking.melt(id_vars=["index"], var_name="Date", value_name="Rank"), x='Date', y='Rank', color='index')
installation_plot

In [157]:
energy_usage_plot = px.line(YoY_Energy_Sales_Ranking.melt(id_vars=["index"], var_name="Date", value_name="Rank"), x='Date', y='Rank', color='index')
energy_usage_plot

In [158]:
energy_cost_plot = px.line(YoY_Energy_Cost_Ranking.melt(id_vars=["index"], var_name="Date", value_name="Rank"), x='Date', y='Rank', color='index')
energy_cost_plot

In [21]:
zip_codes["Zip Codes"] = zip_codes["zip"].apply(lambda zip_codes: map_zip(zip_codes))
zip_codes = zip_codes[['Zip Codes','primary_city','state','latitude','longitude']]

In [22]:
economic_df1 = zip_codes.merge(installation_cost, how='left', left_on='state', right_on='index')
economic_df1 = economic_df1[['Zip Codes','primary_city','state','latitude','longitude','Rank','Weight Score']]
economic_df1 = economic_df1.rename({'Rank':'Installation Cost Rank', 'Weight Score':'Installation Cost Weighted Score'}, axis=1)

economic_df2 = economic_df1.merge(res_cost_prediction, how='left', left_on='state', right_on='index')
economic_df2 = economic_df2[['Zip Codes','primary_city','state','latitude','longitude','Installation Cost Rank','Installation Cost Weighted Score','Rank','Weight Score']]
economic_df2 = economic_df2.rename({'Rank':'Energy Cost Rank', 'Weight Score':'Energy Cost Weighted Score'}, axis=1)

economic_df = economic_df2.merge(res_sales_prediction, how='left', left_on='state', right_on='index')
economic_df = economic_df[['Zip Codes','primary_city','state','latitude','longitude','Installation Cost Rank','Installation Cost Weighted Score','Energy Cost Rank','Energy Cost Weighted Score','Rank','Weight Score']]
economic_df = economic_df.rename({'Rank':'Energy Usage Rank', 'Weight Score':'Energy Usage Weighted Score'}, axis=1)

In [23]:
not_state = ['AA', 'AE', 'AP', 'GU', 'PW', 'FM', 'MP', 'MH', 'AS', 'PR', 'VI']
economic_ranking_df = economic_df[~economic_df['state'].isin(not_state)]

In [24]:
economic_ranking_df['Installation Cost Rank'].fillna((economic_ranking_df['Installation Cost Rank'].quantile(0.75)), inplace=True)
economic_ranking_df['Installation Cost Weighted Score'].fillna((economic_ranking_df['Installation Cost Weighted Score'].quantile(0.75)), inplace=True)

In [25]:
economic_ranking_df['Total Rank'] = economic_ranking_df[['Installation Cost Rank', 'Energy Cost Rank', 'Energy Usage Rank']].mean(axis=1)
economic_ranking_df['Total Weighted Score'] = economic_ranking_df[['Installation Cost Weighted Score', 'Energy Cost Weighted Score', 'Energy Usage Weighted Score']].sum(axis=1)
economic_ranking_df = economic_ranking_df.sort_values('Total Weighted Score', ascending=True).round(2)

In [26]:
economic_ranking_df

,Zip Codes,primary_city,state,latitude,longitude,Installation Cost Rank,Installation Cost Weighted Score,Energy Cost Rank,Energy Cost Weighted Score,Energy Usage Rank,Energy Usage Weighted Score,Total Rank,Total Weighted Score
2050,05860,Orleans,VT,44.81,-72.20,2.0,0.6,3.0,1.5,4.0,0.8,3.00,2.9
1957,05669,Roxbury,VT,44.10,-72.73,2.0,0.6,3.0,1.5,4.0,0.8,3.00,2.9
1956,05667,Plainfield,VT,44.28,-72.43,2.0,0.6,3.0,1.5,4.0,0.8,3.00,2.9
1955,05666,North Montpelier,VT,44.25,-72.48,2.0,0.6,3.0,1.5,4.0,0.8,3.00,2.9
1954,05665,North Hyde Park,VT,44.66,-72.57,2.0,0.6,3.0,1.5,4.0,0.8,3.00,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41432,97454,Marcola,OR,44.21,-122.80,24.0,7.2,51.0,25.5,20.0,4.0,31.67,36.7
41431,97453,Mapleton,OR,44.00,-123.84,24.0,7.2,51.0,25.5,20.0,4.0,31.67,36.7
41430,97452,Lowell,OR,43.87,-122.75,24.0,7.2,51.0,25.5,20.0,4.0,31.67,36.7
41436,97458,Myrtle Point,OR,43.06,-124.03,24.0,7.2,51.0,25.5,20.0,4.0,31.67,36.7


In [160]:
fig = px.choropleth(economic_ranking_df,  # Input Pandas DataFrame
                    locations="state",  # DataFrame column with locations
                    color="Total Weighted Score",  # DataFrame column with color values
                    hover_name="state", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
fig.update_layout(
    title_text = 'State Rankings', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()

In [28]:
#installation_cost.to_csv('installation_cost.csv')
#res_sales_prediction.to_csv('residential_energy_usage.csv')
#res_cost_prediction.to_csv('residential_energy_cost.csv')
#economic_df.to_csv('economic_ranking_dataframe.csv')